In [2]:
import json
import requests
import numpy as np
import pandas as pd

In [2]:
import functools

# logging, timing, arguments

In [4]:
def extract_weather_data(api_url: str):
    response = requests.get(url=api_url)
    response.raise_for_status()
    return response.json()

In [5]:
# convert the data into an hourly data frame (or timestamped by time)
# each row is a time, and all of the hourly attributes at that time
def transform_weather_data(response_data: dict):
    '''Takes an open-meteo api response json in form of a python dict'''
    # load response_data into a raw dataframe
    df = pd.DataFrame.from_dict(data=response_data)
    
    columns_to_extract = df.index

    hourly_data = {
        key: df.at[key, 'hourly']
        for key in columns_to_extract
    }

    hourly_df = pd.DataFrame(data=hourly_data)

    # parse local time into localized datetime
    hourly_df['time'] = pd.to_datetime(hourly_df['time'])
    hourly_df['time'] = hourly_df['time'].dt.tz_localize('America/Los_Angeles')
    
    # calc utc_time from 
    hourly_df['utc_time'] = hourly_df['time'].dt.tz_convert('UTC')
    
    # set utc_time as index
    hourly_df.set_index('utc_time', inplace=True)

    return hourly_df

In [18]:
# pull data from open-meteo api
try:
    # get most recent (current day - 2) hourly data: temp(2m), rel_humid(2m), apparent_temp, uv_index, is_day_or_night
    # location: san francisco
    # timezone: americas/los angeles (gmt - 7/8)
    # BUT store using utc timestamp
    open_meteo_endpoint ='https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=37.7749&longitude=-122.4194&start_date=2025-07-16&end_date=2025-07-16&hourly=temperature_2m,apparent_temperature,uv_index,is_day,relative_humidity_2m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit'
    # open_meteo_endpoint ='https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=37.7749&longitude=-122.4194&start_date=2025-07-16&end_date=2025-07-16&hourly=temperature_2m,apparent_temperature,uv_index,is_day,relative_humidity_2m'
    response_data = extract_weather_data(api_url=open_meteo_endpoint)
except Exception as e:
    print(f'Error: {e}')


In [19]:
# df = pd.DataFrame.from_dict(data=response_data)
flattened_df = pd.json_normalize(data=response_data)
    
display(flattened_df)

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,hourly_units.time,hourly_units.temperature_2m,hourly_units.apparent_temperature,hourly_units.uv_index,hourly_units.is_day,hourly_units.relative_humidity_2m,hourly.time,hourly.temperature_2m,hourly.apparent_temperature,hourly.uv_index,hourly.is_day,hourly.relative_humidity_2m
0,37.763283,-122.41286,1.106381,-25200,America/Los_Angeles,GMT-7,18.0,iso8601,°F,°F,,,%,"[2025-07-16T00:00, 2025-07-16T01:00, 2025-07-1...","[55.9, 55.3, 54.6, 55.6, 55.1, 53.8, 54.3, 55....","[52.9, 51.9, 51.6, 52.2, 52.2, 50.6, 52.4, 53....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.8,...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[90, 89, 92, 88, 89, 89, 90, 91, 90, 89, 88, 8..."


In [47]:
hourly_data = {
    key: flattened_df.loc[0, key] for key in flattened_df.columns if "hourly." in key
}

# put latitude, longitude in each hourly data point
hourly_data["latitude"] = flattened_df.loc[0, "latitude"]
hourly_data["longitude"] = flattened_df.loc[0, "longitude"]

hourly_df = pd.DataFrame.from_dict(hourly_data)

# rename flattened columns strip 'hourly.' prefix, add units to column names
hourly_df = hourly_df.rename(
    columns={
        "hourly.temperature_2m": "temperature_2m_f",
        "hourly.time": "local_time",
        "hourly.apparent_temperature": "apparent_temperature_f",
        "hourly.is_day": "is_day",
        "hourly.relative_humidity_2m": "relative_humidity_2m_perc",
    }
)

# ENFORCE DATA TYPES AFTER LOADING
DTYPE_MAP = {
    "latitude": float,
    "longitude": float,
    "is_day": bool,
    "temperature_2m_f": float,
    "apparent_temperature_f": float,
    "relative_humidity_2m_perc": int,
}
hourly_df = hourly_df.astype(DTYPE_MAP)

# convert time strings to datetimes with timezone awareness
original_timezone = flattened_df.loc[0, "timezone"]

# parse time string into datetime, then make it timezone aware of the original timezone
hourly_df["local_time"] = pd.to_datetime(
    hourly_df["local_time"]
).dt.tz_localize(original_timezone)

# calc utc_time
hourly_df["utc_time"] = hourly_df["local_time"].dt.tz_convert("UTC")

display(hourly_df)
print(hourly_df.info())

,local_time,temperature_2m_f,apparent_temperature_f,hourly.uv_index,is_day,relative_humidity_2m_perc,latitude,longitude,utc_time
0,2025-07-16 00:00:00-07:00,55.9,52.9,0.00,False,90,37.763283,-122.41286,2025-07-16 07:00:00+00:00
1,2025-07-16 01:00:00-07:00,55.3,51.9,0.00,False,89,37.763283,-122.41286,2025-07-16 08:00:00+00:00
2,2025-07-16 02:00:00-07:00,54.6,51.6,0.00,False,92,37.763283,-122.41286,2025-07-16 09:00:00+00:00
3,2025-07-16 03:00:00-07:00,55.6,52.2,0.00,False,88,37.763283,-122.41286,2025-07-16 10:00:00+00:00
4,2025-07-16 04:00:00-07:00,55.1,52.2,0.00,False,89,37.763283,-122.41286,2025-07-16 11:00:00+00:00
5,2025-07-16 05:00:00-07:00,53.8,50.6,0.00,False,89,37.763283,-122.41286,2025-07-16 12:00:00+00:00
6,2025-07-16 06:00:00-07:00,54.3,52.4,0.00,False,90,37.763283,-122.41286,2025-07-16 13:00:00+00:00
7,2025-07-16 07:00:00-07:00,55.2,53.4,0.15,True,91,37.763283,-122.41286,2025-07-16 14:00:00+00:00
8,2025-07-16 08:00:00-07:00,56.2,54.4,0.80,True,90,37.763283,-122.41286,2025-07-16 15:00:00+00:00
9,2025-07-16 09:00:00-07:00,56.9,55.1,1.80,True,89,37.763283,-122.41286,2025-07-16 16:00:00+00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype                              
---  ------                     --------------  -----                              
 0   local_time                 24 non-null     datetime64[ns, America/Los_Angeles]
 1   temperature_2m_f           24 non-null     float64                            
 2   apparent_temperature_f     24 non-null     float64                            
 3   hourly.uv_index            24 non-null     float64                            
 4   is_day                     24 non-null     bool                               
 5   relative_humidity_2m_perc  24 non-null     int64                              
 6   latitude                   24 non-null     float64                            
 7   longitude                  24 non-null     float64                            
 8   utc_time                   24 non-null     datetime64

In [30]:
hourly_cols = [col for col in flattened_df.columns if col.startswith("hourly.")]
hourly_data = flattened_df[hourly_cols].copy()

# put latitude, longitude in each hourly data point
hourly_data["latitude"] = flattened_df["latitude"]
hourly_data["longitude"] = flattened_df["longitude"]

# rename flattened columns strip 'hourly.' prefix, add units to column names
column_rename_map = {
    "hourly.temperature_2m": "temperature_2m_f",
    "hourly.time": "local_time",
    "hourly.apparent_temperature": "apparent_temperature_f",
    "hourly.is_day": "is_day",
    "hourly.relative_humidity_2m": "relative_humidity_2m_perc",
}
hourly_df = hourly_data.rename(columns=column_rename_map)

display(hourly_df)

# ENFORCE DATA TYPES AFTER LOADING
# convert is_day 0/1 into boolean
hourly_df["is_day"] = hourly_df["is_day"].astype(bool)

# parse time string into datetime, then make it timezone aware of the original timezone
original_timezone = flattened_df["timezone"].iloc[0]
hourly_df["local_time"] = pd.to_datetime(
    hourly_df["local_time"], utc=False
).dt.tz_localize(original_timezone)

# calc utc_time
hourly_df["utc_time"] = hourly_df["local_time"].dt.tz_convert("UTC")

display(hourly_df)
print(hourly_df.info())

,local_time,temperature_2m_f,apparent_temperature_f,hourly.uv_index,is_day,relative_humidity_2m_perc,latitude,longitude
0,"[2025-07-16T00:00, 2025-07-16T01:00, 2025-07-1...","[55.9, 55.3, 54.6, 55.6, 55.1, 53.8, 54.3, 55....","[52.9, 51.9, 51.6, 52.2, 52.2, 50.6, 52.4, 53....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.8,...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[90, 89, 92, 88, 89, 89, 90, 91, 90, 89, 88, 8...",37.763283,-122.41286


TypeError: <class 'list'> is not convertible to datetime, at position 0

In [55]:
df = pd.DataFrame.from_dict(data=response_data)

# convert the data into an hourly data frame (or timestamped by time)
# each row is a time, and all of the hourly attributes at that time

columns_to_extract = df.index

hourly_data = {
    key: df.at[key, 'hourly']
    for key in columns_to_extract
}

hourly_df = pd.DataFrame(data=hourly_data)
# parse local time into localized datetime
hourly_df['time'] = pd.to_datetime(hourly_df['time'])
hourly_df['time'] = hourly_df['time'].dt.tz_localize('America/Los_Angeles')
# calc utc_time
hourly_df['utc_time'] = hourly_df['time'].dt.tz_convert('UTC')

# drop localized datetime
hourly_df.drop(columns=['time'], inplace=True)

hourly_df

,temperature_2m,apparent_temperature,uv_index,is_day,relative_humidity_2m,utc_time
0,13.3,11.6,0.00,0,90,2025-07-16 07:00:00+00:00
1,13.0,11.0,0.00,0,89,2025-07-16 08:00:00+00:00
2,12.6,10.9,0.00,0,92,2025-07-16 09:00:00+00:00
3,13.1,11.2,0.00,0,88,2025-07-16 10:00:00+00:00
4,12.9,11.2,0.00,0,89,2025-07-16 11:00:00+00:00
5,12.1,10.3,0.00,0,89,2025-07-16 12:00:00+00:00
6,12.4,11.3,0.00,0,90,2025-07-16 13:00:00+00:00
7,12.9,11.9,0.15,1,91,2025-07-16 14:00:00+00:00
8,13.5,12.5,0.80,1,90,2025-07-16 15:00:00+00:00
9,13.9,12.8,1.80,1,89,2025-07-16 16:00:00+00:00


In [ ]:
hourly_df.dtypes

# Load into Postgres

In [ ]:
import psycopg2

# use context managers in finalized .py files
conn = psycopg2.connect(
    database='weather_db',
    user='postgres',
    password='example',
    host='localhost',
    port=5432
)

cur = conn.cursor()
# cur.execute('DROP TABLE sf_hourly_weather')


In [ ]:
from io import StringIO

# create table if doesnt exist
cur.execute('''
CREATE TABLE IF NOT EXISTS sf_hourly_weather (
    temperature_2m_f REAL,
    apparent_temperature_f REAL,
    uv_index REAL,
    is_day BOOLEAN,
    relative_humidity_2m_perc REAL,
    utc_time TIMESTAMPTZ PRIMARY KEY
)
''')

with StringIO() as buffer:
    hourly_df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    cur.copy_from(buffer, table='sf_hourly_weather', sep=',')

In [ ]:
print(type(conn))

In [ ]:
cur.execute('select * from sf_hourly_weather')
result_list = cur.fetchall()
columns = [col.name for col in cur.description]
query_df = pd.DataFrame(data=result_list, columns=columns)

# query_df = pd.read_sql(sql="select utc_time at time zone 'America/Los_Angeles', * from sf_hourly_weather", con=conn)
query_df